In [1]:
from utils_kdd99 import *
print_version()

python:      3.10.11
sklearn:     1.2.2
tensorflow:  2.12.0
keras:       2.12.0
numpy:       1.23.5
pandas:      1.5.3


In [13]:
X, y = load_data(use_full_dataset=False, standard_scale=True, verbose=0, )
X.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,-0.067792,-0.002879,0.138664,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,2.39698,-0.005679,...,-3.451536,-1.694315,0.599396,-0.282867,-1.022077,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464
1,-0.067792,-0.002820,-0.011578,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,2.39698,-0.005679,...,-3.297085,-1.600011,0.599396,-0.282867,-1.146737,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464
2,-0.067792,-0.002824,0.014179,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,2.39698,-0.005679,...,-3.142633,-1.505707,0.599396,-0.282867,-1.188291,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464
3,-0.067792,-0.002840,0.014179,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,2.39698,-0.005679,...,-2.988182,-1.411403,0.599396,-0.282867,-1.188291,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464
4,-0.067792,-0.002842,0.035214,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,2.39698,-0.005679,...,-2.833731,-1.317100,0.599396,-0.282867,-1.209067,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464


In [3]:
X_25features = X.drop(labels=['hot', 'num_compromised', 'num_file_creations', 'num_outbound_cmds', 'is_host_login', 'srv_count', 'srv_serror_rate', 'srv_rerror_rate', 'same_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_diff_srv_rate'], axis=1)


In [14]:
y = y.map(lambda x: attack_label_class[x]).map(lambda x: correspondences[x])
# x_train, x_test, y_train, y_test = train_test_split(X_25features, y, test_size=0.33, random_state=RANDOM_SEED, stratify=y)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=RANDOM_SEED, stratify=y)


In [5]:
# x_train.to_pickle("models/kdd99_features/x_train-drop_25_df.pkl")
# x_test.to_pickle("models/kdd99_features/x_test-drop_25_df.pkl")


In [5]:
# ラベルのリマッピング
label_mapping = {0: 0, 2: 1, 3: 2, 4: 3}

In [6]:
y_test.head()

38204     1
176442    0
384838    0
261369    0
17533     1
Name: true_label, dtype: int64

In [15]:
x_train_dropped = x_train[y_train != 1]
x_test_dropped = x_test[y_test != 1]

y_train_dropped: pd.Series = y_train[y_train != 1]
y_test_dropped: pd.Series = y_test[y_test != 1]
y_train_mapped = y_train_dropped.map(label_mapping)
y_test_mapped = y_test_dropped.map(y_train_mapped)


In [16]:
x_test_dropped.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
176442,-0.067792,-0.002017,-0.026287,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,-0.417192,-0.005679,...,0.347967,0.625558,0.599396,-0.282867,0.827048,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464
384838,-0.067792,-0.003062,-0.026287,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,-0.417192,-0.005679,...,0.347967,-1.769758,-1.834994,0.357813,-1.250621,-0.158629,2.163063,2.162027,-0.25204,-0.249464
261369,-0.067792,-0.002017,-0.026287,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,-0.417192,-0.005679,...,0.347967,0.625558,0.599396,-0.282867,0.827048,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464
400731,-0.067792,-0.002535,-0.026287,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,-0.417192,-0.005679,...,0.347967,0.625558,0.599396,-0.282867,0.827048,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464
285355,-0.067792,-0.002017,-0.026287,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,-0.417192,-0.005679,...,0.347967,0.625558,0.599396,-0.282867,0.827048,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464


In [19]:
x_train_dropped.shape

(265818, 38)

In [20]:
y_train_mapped.to_pickle("models/kdd99_features/y_train_dropped_mapped_series.pkl")
y_test_mapped.to_pickle("models/kdd99_features/y_test_dropped_mapped_series.pkl")
x_train_dropped.to_pickle("models/kdd99_features/x_train_anomaly_df.pkl")
x_test_dropped.to_pickle("models/kdd99_features/x_test_anomaly_df.pkl")